# archive to folder

In [1]:
# nuclio: ignore
import nuclio

In [2]:
import os
import zipfile
import urllib.request
import tarfile
import json

from mlrun.execution import MLClientCtx
from mlrun.datastore import DataItem

from typing import Union

def open_archive(
    context: MLClientCtx, 
    archive_url: DataItem = "",
    subdir: str = "content",
    key: str = "content"
):
    """Open a file/object archive into a target directory
    
    Currently supports zip and tar.gz
    
    :param context:      function execution context
    :param archive_url:  url of archive file
    :param subdir:       path within artifact store where extracted files
                         are stored
    :param key:          key of archive contents in artifact store
    """
    target_dir = os.path.join(context.artifact_path, subdir)
    os.makedirs(target_dir, exist_ok=True)
    
    archive_url = str(archive_url)
    if archive_url.endswith("gz"):
        with tarfile.open(archive_url, mode="r|gz") as ref:
            ref.extractall(target_dir)
    elif archive_url.endswith("zip"):
        with zipfile.ZipFile(archive_url, "r") as ref:
            ref.extractall(target_dir)
    else:
        raise ValueError(f'unsupported archive type in {archive_url}')
    
    context.log_artifact(key, local_path=subdir)

In [3]:
# nuclio: end-code

### mlconfig

In [4]:
from mlrun import mlconf

In [5]:
mlconf.dbpath = mlconf.dbpath or './'
mlconf.dbpath

'http://mlrun-api:8080'

In [6]:
vcs_branch = 'development'
base_vcs = f'https://raw.githubusercontent.com/mlrun/functions/{vcs_branch}/'

mlconf.hub_url = mlconf.hub_url or base_vcs + f'{name}/function.yaml'
mlconf.hub_url

'/User/repos/functions/{name}/function.yaml'

In [7]:
import os
mlconf.artifact_path = mlconf.artifact_path or f'{os.environ["V3IO_HOME"]}/artifacts'
mlconf.artifact_path

'/User/artifacts'

### save

In [8]:
from mlrun import code_to_function 
# create job function object from notebook code
fn = code_to_function("open_archive", kind="job", with_doc=True,
                      handler=open_archive, 
                      image="mlrun/ml-base:0.4.6")

# add metadata (for templates and reuse)
fn.spec.default_handler = "open_archive"
fn.spec.description = "Open a file/object archive into a target directory"
fn.metadata.categories = ["fileutils", "retrieve"]
fn.metadata.labels = {"author": "yaronh"}

fn.save()
fn.export("function.yaml")

[mlrun] 2020-04-25 22:49:35,125 saving function: open-archive, tag: latest
[mlrun] 2020-04-25 22:49:35,150 function spec saved to path: function.yaml


## tests

In [9]:
from mlrun import import_function, run_local

func = import_function("hub://open_archive")

if "V3IO_HOME" in list(os.environ):
    from mlrun import mount_v3io
    func.apply(mount_v3io())
else:
    # is you set up mlrun using the instructions at https://github.com/mlrun/mlrun/blob/master/hack/local/README.md
    from mlrun.platforms import mount_pvc
    func.apply(mount_pvc('nfsvol', 'nfsvol', '/home/joyan/data'))

### tar

In [12]:
run_local(
    handler=open_archive, 
    inputs={'archive_url': "https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz"},
    artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-25 22:52:54,081 starting run mlrun-45dc13-open_archive uid=9526b91ba1ef4f9b8fdf24818b0f9433  -> http://mlrun-api:8080
[mlrun] 2020-04-25 22:52:54,109 downloading https://fpsignals-public.s3.amazonaws.com/catsndogs.tar.gz to local tmp
[mlrun] 2020-04-25 22:53:06,258 log artifact content at /User/artifacts/content, size: None, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...8b0f9433,0,Apr 25 22:52:54,completed,mlrun-45dc13-open_archive,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-hwhxd,archive_url,,,content


to track results use .show() or .logs() or in CLI: 
!mlrun get run 9526b91ba1ef4f9b8fdf24818b0f9433 --project default , !mlrun logs 9526b91ba1ef4f9b8fdf24818b0f9433 --project default
[mlrun] 2020-04-25 22:53:06,293 run executed, status=completed


### zip

In [13]:
run_local(
    handler=open_archive, 
    inputs={'archive_url': 'http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip'},
    artifact_path=mlconf.artifact_path)

[mlrun] 2020-04-25 22:53:06,325 starting run mlrun-d64bea-open_archive uid=507a06a0b9e84999862fa497fc09f6df  -> http://mlrun-api:8080
[mlrun] 2020-04-25 22:53:06,353 downloading http://iguazio-sample-data.s3.amazonaws.com/catsndogs.zip to local tmp
[mlrun] 2020-04-25 22:53:14,763 log artifact content at /User/artifacts/content, size: None, db: Y



project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...fc09f6df,0,Apr 25 22:53:06,completed,mlrun-d64bea-open_archive,v3io_user=adminkind=handlerowner=adminhost=jupyter-5859859b4f-hwhxd,archive_url,,,content


to track results use .show() or .logs() or in CLI: 
!mlrun get run 507a06a0b9e84999862fa497fc09f6df --project default , !mlrun logs 507a06a0b9e84999862fa497fc09f6df --project default
[mlrun] 2020-04-25 22:53:14,799 run executed, status=completed
